In [1]:
# Standard library imports
import csv
import itertools
import os
import os.path
import random
from glob import glob
from pathlib import Path
import zlib
import importlib

# Third-party imports
import cv2
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import skimage
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from skimage.transform import resize
from sklearn import model_selection
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import (KFold, StratifiedKFold, cross_val_score,
                                     learning_curve, train_test_split)
from sklearn.utils import class_weight, shuffle
from tqdm import tqdm

# Keras-specific imports
import keras
from keras import backend as K
from keras import layers, models, optimizers
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import (Activation, AveragePooling2D, BatchNormalization,
                          Conv2D, Dense, Dropout, Flatten, Lambda, MaxPool2D,
                          MaxPooling2D)
from keras.models import Model, Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.utils import to_categorical

# Custom imports
import fit
import evaluate
import viz

%matplotlib inline

KeyboardInterrupt: 

In [ ]:
X_test_pca = np.load("/Users/mnorton/Desktop/College/Grad Victory Lap/CMSE 890/data/X_test_pca.npy")
y_test = np.load("/Users/mnorton/Desktop/College/Grad Victory Lap/CMSE 890/data/y_test.npy")
y_testHot = np.load("/Users/mnorton/Desktop/College/Grad Victory Lap/CMSE 890/data/y_testHot.npy")
X_test = np.load("/Users/mnorton/Desktop/College/Grad Victory Lap/CMSE 890/data/X_test.npy")

In [ ]:
model = tf.keras.models.load_model('assets/models/ASL_DNN_3D.h5')

In [ ]:
# Predict the classes for the 3D test set
y_pred3D = model.predict(X_test)

# If the predictions are probabilities (as with some models), 
# you'll need to convert to predicted class labels by taking the argmax
if y_pred3D.ndim > 1:
    y_pred3D = np.argmax(y_pred3D, axis=1)

# Ensure 'y_test' is not one-hot encoded. If it is, convert it:
if y_test.ndim > 1:
    y_test3D = np.argmax(y_test, axis=1)

# Calculate the accuracy score
accuracy = accuracy_score(y_test3D, y_pred3D)

print(f"Accuracy score: {accuracy}")


In [ ]:

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred3D)

print(f"Accuracy score: {accuracy}")
